# Mathematical Underpinnings - Lab 6

In [1]:
from sklearn.metrics import mutual_info_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm

## Useful functions

In [2]:
def discetize_2bins(X):
    X_discrete = 1*(X >= 0)
    return X_discrete

In [3]:
def conditional_permutation(X, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    X_b = np.zeros(n)

    for i in range(n_z_values):

        z_value_tmp = z_values[i]

        X_b[Z == z_value_tmp] = np.random.permutation(X[Z == z_value_tmp])

    return X_b

In [4]:
def conditional_mutual_information(X, Y, Z):

    z_values = np.unique(Z)
    n_z_values = len(z_values)
    n = len(Z)

    cmi = 0

    for i in range(n_z_values):

        z_value_tmp = z_values[i]
        z_condition = (Z == z_value_tmp)

        X_z = X[z_condition]
        Y_z = Y[z_condition]

        mi_XY_z = mutual_info_score(X_z, Y_z)
        p_z = np.sum(z_condition)/n

        cmi += p_z*mi_XY_z

    return cmi

In [5]:
# II(X;Y;Z)
def interaction_information(X, Y, Z):
    return conditional_mutual_information(X, Y, Z) - mutual_info_score(X, Y)

In [6]:
# II(X;Y;Z1;Z2)
def interaction_information2(X, Y, Z1, Z2):
    # Zs are binary
    Z_1_and_2 = 2*Z2 + Z1
    return interaction_information(X, Y, Z_1_and_2) - interaction_information(X, Y, Z1) - interaction_information(X, Y, Z2)

## Task 1

In [7]:
def secmi2(X, Y, Z):
    n_col_Z = Z.shape[1]
    secmi = mutual_info_score(X, Y)
    for i in range(n_col_Z):
        secmi += interaction_information(X, Y, Z[:, i])
    return secmi

def secmi3(X, Y, Z):
    n_col_Z = Z.shape[1]
    secmi = secmi2(X, Y, Z)
    for i in range(n_col_Z):
        for j in range(i):
            secmi += interaction_information2(X, Y, Z[:, i], Z[:, j])
    return secmi


### a)

In [8]:
def cond_indep_test_permutation(X, Y, Z, B, stat):

    n_col_Z = Z.shape[1]
    Z_1dim = np.dot(Z, 2**np.linspace(0, n_col_Z-1, n_col_Z))

    if stat == "cmi":
        stat_value = conditional_mutual_information(X, Y, Z_1dim)
    if stat == "secmi2":
        stat_value = secmi2(X, Y, Z)
    if stat == "secmi3":
        stat_value = secmi3(X, Y, Z)

    condition_p_value = 0
    for b in range(B):
        X_b = conditional_permutation(X, Z_1dim)

        if stat == "cmi":
            stat_value_b = conditional_mutual_information(X_b, Y, Z_1dim)
        if stat == "secmi2":
            stat_value_b = secmi2(X_b, Y, Z)
        if stat == "secmi3":
            stat_value_b = secmi3(X_b, Y, Z)

        if stat_value <= stat_value_b:
            condition_p_value += 1

    p_value = (1 + condition_p_value)/(1 + B)

    return 2*len(X)*stat_value, p_value

### b)

In [9]:
def gen_schema_1(n):
    Y = np.random.normal(size=n)
    Y_disc = discetize_2bins(Y)
    Z1 = np.random.normal(loc=Y_disc/2, size=n)
    Z2 = np.random.normal(loc=Y_disc/2, size=n)
    Z3 = np.random.normal(loc=Y_disc/2, size=n)
    X = np.random.normal(loc=discetize_2bins(Z1)/2, size=n)
    return discetize_2bins(X), Y_disc, discetize_2bins(Z1), discetize_2bins(Z2), discetize_2bins(Z3)

In [11]:
# should be rejected
n_retries = 100
rejected = 0
for _ in range(n_retries):
    X, Y, Z1, Z2, Z3 = gen_schema_1(100)
    Z = np.stack([Z3, Z2], axis=1)
    _, p_val = cond_indep_test_permutation(X, Y, Z, 50, "secmi3")
    rejected += p_val < 0.05
print(rejected/n_retries)

0.06


In [12]:
# should not be rejected
n_retries = 100
rejected = 0
for _ in range(n_retries):
    X, Y, Z1, Z2, Z3 = gen_schema_1(100)
    Z = np.stack([Z1, Z2], axis=1)
    _, p_val = cond_indep_test_permutation(X, Y, Z, 50, "secmi3")
    rejected += p_val < 0.05
print(rejected/n_retries)

0.05


The test doesn't work for small n (like 100), but starts working for bigger sample sizes:

In [13]:
# should be rejected
n_retries = 100
rejected = 0
for _ in range(n_retries):
    X, Y, Z1, Z2, Z3 = gen_schema_1(10000)
    Z = np.stack([Z3, Z2], axis=1)
    _, p_val = cond_indep_test_permutation(X, Y, Z, 50, "secmi3")
    rejected += p_val < 0.05
print(rejected/n_retries)

0.74


### c)

In [15]:
def gen_schema_2(n):
    X = np.random.binomial(1, 0.5, n)
    Z1 = np.random.binomial(1, 0.5, n)
    Z2 = np.random.binomial(1, 0.5, n)
    Z3 = np.random.binomial(1, 0.5, n)
    XOR = (X + Z1 + Z2)%2

    y_probs = np.where(XOR, 0.8, 0.2)
    Y = np.random.binomial(1, y_probs, n)
    return X, Y, Z1, Z2, Z3

In [16]:
# should not be rejected
n_retries = 100
rejected = 0
for _ in range(n_retries):
    X, Y, Z1, Z2, Z3 = gen_schema_2(100)
    Z = np.stack([Z3, Z2], axis=1)
    _, p_val = cond_indep_test_permutation(X, Y, Z, 50, "secmi3")
    rejected += p_val < 0.05
print(rejected/n_retries)

0.04


In [17]:
# should be rejected
n_retries = 100
rejected = 0
for _ in range(n_retries):
    X, Y, Z1, Z2, Z3 = gen_schema_2(100)
    Z = np.stack([Z1, Z2], axis=1)
    _, p_val = cond_indep_test_permutation(X, Y, Z, 50, "secmi3")
    rejected += p_val < 0.05
print(rejected/n_retries)

1.0


In this case the test works even on small samples

## Task 2
 
in R